# 라이브러리

In [16]:
import numpy as np
import torch
from __future__ import print_function
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F
import time

from google.colab import drive
drive.mount('/content/drive')

device = 'cuda' if cuda.is_available() else 'cpu'
path = "/content/drive/MyDrive/21_winter/CIFAR-10/cifar-10-batches-py"
batch_size = 32

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 데이터 불러오기

In [17]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
data_list = ["/data_batch_" + str(i) for i in range(2,6)]
train_data = unpickle(path + "/data_batch_1")
test_data = unpickle(path + "/test_batch")

train_data_img = train_data[b'data']
train_data_labels = train_data[b'labels']
for i in data_list:
  train_data = unpickle(path + i)
  train_data_img = np.concatenate((train_data_img,train_data[b'data']))
  train_data_labels += train_data[b'labels']


# 데이터 준비

In [18]:
class GH_Dataset(data.Dataset): 
    def __init__(self,X,Y):
        self.x_data = torch.from_numpy(X).type(dtype=torch.float32).resize_((10000,3,32,32))
        self.y_data = torch.tensor(Y).resize_(10000,1)

    def __len__(self): 
        return len(self.x_data)
    
    def __getitem__(self, idx): 
        x = self.x_data[idx]
        y = self.y_data[idx]
        return x, y

train_dataset = GH_Dataset(train_data_img,train_data_labels)
train_loader = data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

test_dataset = GH_Dataset(test_data[b'data'],test_data[b'labels'])
test_loader = data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

# 모델

In [19]:
class GH(nn.Module):
    def __init__(self):
        super(GH,self).__init__()
        self.l1 = nn.Conv2d(3 ,64,3, padding = 1)
        self.l2 = nn.Conv2d(64,64,3, padding = 1)

        self.l3 = nn.Conv2d(64,128,3, padding = 1)
        self.l4 = nn.Conv2d(128,128,3, padding = 1)

        self.l5 = nn.Conv2d(128,256,3, padding = 1)
        self.l6 = nn.Conv2d(256,256,3, padding = 1)
        
        self.l7 = nn.Conv2d(256,512,3, padding = 1)
        self.l8 = nn.Conv2d(512,512,3, padding = 1)
        self.l9 = nn.Conv2d(512,512,3, padding = 1)

        self.l10 = nn.Linear(2048,2048)   
        self.l11 = nn.Linear(2048,2048)
        self.l12 = nn.Linear(2048,10)
    
    def forward(self, x):      #(n,3,32,32)
        x = F.relu(self.l1(x)) #(n,64,32,32)
        x = F.relu(self.l2(x)) #(n,64,32,32)
        x = F.max_pool2d(x,2)  #(n,64,16,16)

        x = F.relu(self.l3(x)) #(n,128,16,16)
        x = F.relu(self.l4(x)) #(n,128,16,16)
        x = F.max_pool2d(x,2)  #(n,128, 8, 8)

        x = F.relu(self.l5(x)) #(n,256, 8, 8)
        x = F.relu(self.l6(x)) #(n,256, 8, 8)
        x = F.max_pool2d(x,2)  #(n,256, 4, 4)

        x = F.relu(self.l7(x)) #(n,512, 4, 4)
        x = F.relu(self.l8(x)) #(n,512, 4, 4)
        x = F.relu(self.l8(x)) #(n,512, 4, 4)
        x = F.max_pool2d(x,2)  #(n,512, 2, 2)

        x = x.view(-1,2048)    #(n,2048)
        x = F.relu(self.l10(x)) #(n,2048)
        x = F.relu(self.l11(x)) #(n,2048)
        x = F.relu(self.l12(x)) #(n,10) 
        return x               #(n,10)
    
model = GH()
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00005)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, torch.squeeze(target))
        loss.backward()
        optimizer.step()
        if batch_idx %100000 == 0:
            print('==================\nTrain Epoch : {} | Batch Status : {}/{} ({:.0f}%) | Loss : {:.6f}'.format(
                epoch, batch_idx*len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, torch.squeeze(target)).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    test_loss /= len(test_loader.dataset)
    print(f'Test set: Average loss : {test_loss:.4f}, Accuracy : {correct}/{len(test_loader.dataset)}'
          f'({100. * correct / len(test_loader.dataset):.0f}%)')

# 모델 학습

In [20]:
if __name__ == '__main__':
    since = time.time()
    for epoch in range(1, 10):
        epoch_start = time.time()
        train(epoch)
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Training time: {m:.0f}m {s:.0f}s')
        
        test()
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Tesing time: {m:.0f}m {s:.0f}s')
        
    m, s = divmod(time.time() - epoch_start, 60)
    print(f'Total time : {m:.0f}m {s: .0f}s \nModel was trained on {device}!')

Train Epoch : 1 | Batch Status : 0/10000 (0%) | Loss : 2.304150
Training time: 0m 6s
Test set: Average loss : 0.0707, Accuracy : 1610/10000(16%)
Tesing time: 0m 8s
Train Epoch : 2 | Batch Status : 0/10000 (0%) | Loss : 2.298559
Training time: 0m 6s
Test set: Average loss : 0.0707, Accuracy : 1609/10000(16%)
Tesing time: 0m 8s
Train Epoch : 3 | Batch Status : 0/10000 (0%) | Loss : 2.265885
Training time: 0m 6s
Test set: Average loss : 0.0677, Accuracy : 2367/10000(24%)
Tesing time: 0m 8s
Train Epoch : 4 | Batch Status : 0/10000 (0%) | Loss : 2.229309
Training time: 0m 6s
Test set: Average loss : 0.0664, Accuracy : 2618/10000(26%)
Tesing time: 0m 8s
Train Epoch : 5 | Batch Status : 0/10000 (0%) | Loss : 2.233996
Training time: 0m 6s
Test set: Average loss : 0.0657, Accuracy : 2678/10000(27%)
Tesing time: 0m 8s
Train Epoch : 6 | Batch Status : 0/10000 (0%) | Loss : 2.030678
Training time: 0m 6s
Test set: Average loss : 0.0654, Accuracy : 2735/10000(27%)
Tesing time: 0m 8s
Train Epoch : 7 

In [ ]:
"""
이미지 데이터의 크기가 32*32 사이즈이므로 pool을 여러번 적용하게 되면서 데이터의 손실이 일어나 예측이 낮아진것이라 생각됨
"""